In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from keras import models,layers
from keras import regularizers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.externals import joblib
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from keras.models import load_model
import os
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
class Autometion():
    def __init__(self, score, arg1, arg2 = None, arg3 = None, arg4 = None, arg5 = None, arg6 = None, arg7 = None, arg8 = None):
        self.score = score
        self.arg = [arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8]
        i = 7
        
        # 引数が無ければ削除
        for ar in reversed(self.arg):
            if ar == None:
                self.arg.pop(i)
            i -= 1
                
        

    def preparation(self, train, test):
    
        # trainとtestを結合する前に判別フラグを立てる
        train['train_flg'] = 1
        test['train_flg'] = 0

        # testに答えの列を追加
        test[train.columns[1]] = 9

        # trainとtestを結合
        train_test_combine = pd.concat([train,test],axis=0)

        # 削除する行名と列名の格納するリストを定義
        delete_row = []
        delete_column = []

        # 欠損値の割合を計算
        percent = 100 * train_test_combine.isnull().sum()/ len(train_test_combine)      

        # 特徴量の数だけループ
        for i in range(percent.size):
            # object型かつカラムに30種類以上のデータが含まれる、または、欠損値が全体の15％以上ある場合列を削除
            if (train_test_combine[percent.index[i]].dtype == ('object') 
            and train_test_combine[percent.index[i]].nunique() >= 30)or percent[i] >= 15:
                delete_column.append(percent.index[i])
            # 欠損値が存在し、全体の15％以下である場合行を削除
            elif percent[i] > 0 and percent[i] < 15:
                delete_row.append(percent.index[i])

        # 行の削除
        for i in delete_row:
            train_test_combine = train_test_combine[train_test_combine[i].notnull()]
        # 列の削除
        for j in delete_column:
            train_test_combine = train_test_combine.drop([j], axis=1)

        # trainの答えを退避
        train_y = train_test_combine[train_test_combine['train_flg'] == 1][train.columns[1]]
        # testのIDを退避
        test_ID = train_test_combine[train_test_combine['train_flg'] == 0][test.columns[0]]

        # ID, 答えを削除
        train_test_combine = train_test_combine.drop([train.columns[0],train.columns[1]],axis=1) 

        # カテゴリカル変数の特徴量だけを抽出
        categorical_columns = [c for c in train_test_combine.columns if (train_test_combine[c].dtype != ('float64') 
                                                                         and train_test_combine[c].dtype != ('int64') 
                                                                         and train_test_combine[c].dtype != ('bool'))]

        # カテゴリカル変数の種類と個数を出力し、one-hot-encoding
        for col in categorical_columns:
            ohe = pd.get_dummies(train_test_combine[col],drop_first=True)
            train_test_combine = train_test_combine.drop([col],axis=1)
            train_test_combine = pd.concat([train_test_combine,ohe], axis=1)

        train = train_test_combine[train_test_combine['train_flg'] == 1].drop(['train_flg'], axis=1)
        test = train_test_combine[train_test_combine['train_flg'] == 0].drop(['train_flg'], axis=1)

        return train, train_y, test, test_ID
    
    def evaluation(self, x, y, ranking, arg):

        for i in arg:
            if i == 'logisticRegression':
                ranking = self.logistic(x, y, ranking)
            elif i == 'SVC':
                ranking = self.SVCK(x, y, ranking)
            elif i == 'naiveBayes':
                ranking = self.naive(x, y, ranking)
            elif i == 'randomForest':
                ranking = self.randomForest(x, y, ranking)
            elif i == 'xgboost':
                ranking = self.XGboost(x, y, ranking)
            elif i == 'adaboost':
                ranking = self.ADAboost(x, y, ranking)
            elif i == 'neuralNet':
                ranking = self.NeuralNet(x, y, ranking)
            elif i == 'KNeighborsClassifier':
                ranking = self.KNeighber(x, y, ranking)
            else:
                print(i + 'は対応していません')

        return ranking
    
    def logistic(self, x, y, ranking):
        #ロジスティック回帰
        logistic_regression = LogisticRegressionCV(random_state=0, penalty='l2', Cs=10, n_jobs=-1)
        answer = cross_val_score(logistic_regression, x, y, scoring = self.score)
        ranking['ロジスティック回帰'] = np.average(answer)
        return ranking
    
    def SVCK(self, x, y, ranking):
        #サポートベクトルマシン
        svc = SVC(C=1.0,kernel='linear',probability=True,random_state=0)
        answer = cross_val_score(svc, x, y, scoring = self.score)
        ranking['サポートベクトルマシン'] = np.average(answer)
        return ranking
    
    def naive(self, x, y, ranking):
        #ナイーブベイズ
        naive = GaussianNB()
        answer = cross_val_score(naive, x, y, scoring = self.score)
        ranking['ナイーブベイズ'] = np.average(answer)
        return ranking
    
    def randomForest(self, x, y, ranking):
        #ランダムフォレスト
        RF = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=0, n_jobs=-1)
        answer = cross_val_score(RF, x, y, scoring = self.score)
        ranking['ランダムフォレスト'] = np.average(answer)
        return ranking
    
    def XGboost(self, x, y, ranking):
        #Xgboost
        xgboost = xgb.XGBClassifier()
        answer = cross_val_score(xgboost, x, y, scoring = self.score)
        ranking['XGブースト'] = np.average(answer)
        return ranking
    
    def ADAboost(self, x, y, ranking):
        #ADAブースト
        adaboost = AdaBoostClassifier(random_state=0)
        answer = cross_val_score(adaboost, x, y, scoring = self.score)
        ranking['ADAブースト'] = np.average(answer)
        return ranking
    
    def KNeighber(self, x, y, ranking):
        #K近傍法
        Knn = KNeighborsClassifier()
        answer = cross_val_score(Knn, x, y, scoring =self.score)
        ranking['K近傍法'] = np.average(answer)
        return ranking
    
    def NeuralNet(self, x, y, ranking):
        # ニューラルネット
        if self.score != 'accuracy':
            print('NNを使う場合の評価関数はaccuracyのみのため、NNをランキングに含めません。')
            return ranking
        
        (X_train, X_test, y_train, y_test) = train_test_split(x, y, test_size=0.3, random_state=0)

        model = models.Sequential()
        model.add(layers.Dense(units=32, activation='relu', input_shape=(X_train[0].size,)))
        model.add(layers.Dense(units=64, activation='relu'))
        model.add(layers.Dropout(0.2))
        model.add(layers.Dense(units=1, activation='sigmoid'))
        model.compile(loss="binary_crossentropy", optimizer = "rmsprop", metrics=[self.score])

        # 早期終了を行うコールバック関数を設定
        callbacks = [EarlyStopping(monitor="val_loss", patience=2)]

        history = model.fit(X_train, y_train, epochs=20, verbose=0, callbacks = callbacks, batch_size=100) 
        loss, scores = model.evaluate(X_test, y_test)

        ranking['ニューラルネット'] = scores
        return ranking
    
    def result(self, x, y, xt, test_ID, score, ranking):
        # ランキングを表示
        print("評価指標：" + score)
        print()
        print("アルゴリズムランキング")
        j = 1

        for k, i in ranking:
            print(str(j) + "位：" + k,end="")
            print("（" ,end="")
            print(i, end="")
            print("）")
            if j == 1:
                if k == 'ニューラルネット':
                    model = models.Sequential()
                    model.add(layers.Dense(units=32, activation='relu', input_shape=(x[0].size,)))
                    model.add(layers.Dense(units=64, activation='relu'))
                    model.add(layers.Dropout(0.2))
                    model.add(layers.Dense(units=1, activation='sigmoid'))
                    callbacks = [EarlyStopping(monitor="val_loss", patience=2),
                                 ModelCheckpoint(filepath = "best_model.h5",monitor = "val_loss", save_best_only = True)]
                    model.compile(loss="binary_crossentropy", optimizer = "rmsprop", metrics=[score])
                    model.fit(x, y, epochs=20, verbose=0, callbacks = callbacks, batch_size=100,validation_split=0.3) 
                    predicted = model.predict(xt)
                    answer= pd.concat([test_ID, pd.DataFrame(predicted)],axis = 1)
                    if os.path.isfile('best_model.pkl'):
                        os.remove('best_model.pkl')

                else:
                    if k == 'ロジスティック回帰':
                        model = LogisticRegressionCV(random_state=0, penalty='l2', Cs=10, n_jobs=-1).fit(x,y)
                    elif k == 'サポートベクトルマシン':
                        model = SVC(C=1.0,kernel='linear',probability=True,random_state=0).fit(x,y)
                    elif k == 'ナイーブベイズ':
                        model = CalibratedClassifierCV(GaussianNB(), cv=2, method='sigmoid')
                    elif k == 'ランダムフォレスト':
                        model = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=0, n_jobs=-1).fit(x,y)
                    elif k == 'XGブースト':
                        model = xgb.XGBClassifier().fit(x,y)
                    elif k == 'ADAブースト':
                        model = AdaBoostClassifier(random_state=0).fit(x,y)
                    elif k == 'K近傍法':
                        model = KNeighborsClassifier().fit(x,y)


                    joblib.dump(model, "best_model.pkl")
                    predicted = model.predict_proba(xt)
                    answer = pd.concat([test_ID, pd.DataFrame(predicted)[1]],axis = 1)
                    answer.columns = ['ID','predicted']
                    if os.path.isfile('best_model.h5'):
                        os.remove('best_model.h5')

            j += 1

        answer.to_csv("answer.csv",index=False)
        
    def run(self):
        # データの読み込み
        train = pd.read_csv("train.csv")
        test  = pd.read_csv("test.csv")

        # データの準備
        train_x, train_y, test, test_ID = self.preparation(train, test)

        # 正規化
        scaler = StandardScaler()
        train_x = scaler.fit_transform(train_x)
        test = scaler.fit_transform(test)

        #　アルゴリズムの結果を格納するリスト
        ranking = {}

        ranking = self.evaluation(train_x, train_y, ranking, self.arg)

        # アルゴリズムをランキング形式で並べ替え
        ranking = sorted(ranking.items(), key=lambda x:-x[1])

        # 結果
        self.result(train_x, train_y, test, test_ID, self.score, ranking)
        

In [ ]:
# 第一引数
# 評価関数
# accuracy
# precision
# recall
# f1
# ※ニューラルネットを使う場合はaccuracyのみ

# 第2-8引数
# アルゴリズム
# logisticRegression
# SVC
# naiveBayes
# randomForest
# xgboost
# adaboost
# neuralNet
# KNeighborsClassifier
auntomation = Autometion('accuracy','xgboost','neuralNet','randomForest', 'logisticRegression')

# 実行
auntomation.run()